# 第9章: ベクトル空間法 (I)


[enwiki-20150112-400-r10-105752.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2)は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパス[enwiki-20150112-400-r100-10576.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2)を用いよ．



### ファイル解凍
- bzip2 -d (ファイルが上書きされてしまう)
- bunzip2 -k (元のファイルを残す)

In [1]:
%%bash
if [ ! -f 'work/enwiki-20150112-400-r100-10576.txt' ]; then
    bunzip2 -k 'data/enwiki-20150112-400-r10-105752.txt.bz2'
    bunzip2 -k 'data/enwiki-20150112-400-r100-10576.txt.bz2'
    mv 'data/enwiki-20150112-400-r10-105752.txt' 'work'
    mv 'data/enwiki-20150112-400-r100-10576.txt' 'work'
fi

### ファイルサイズ確認

In [2]:
import os 

In [3]:
size10 = os.path.getsize('work/enwiki-20150112-400-r10-105752.txt')
size100 = os.path.getsize('work/enwiki-20150112-400-r100-10576.txt')
print('1/10 サンプリング : {} Mb'.format(size10 / 1024 / 1024))
print('1/100 サンプリング : {} Mb'.format(size100 / 1024 / 1024))

1/10 サンプリング : 710.0323333740234 Mb
1/100 サンプリング : 70.12497138977051 Mb


### 中身確認

In [4]:
!head -n 3 'work/enwiki-20150112-400-r100-10576.txt'
!echo '\n...\n'
!tail -n 5 'work/enwiki-20150112-400-r100-10576.txt'

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.

...

Stanley Hall has a picturesque external form which reflects the stages of its construction and particular interests of its former owners, and contains some finely crafted internal elements. It is a fine example of the work of Brisbane architect GHM Addison.
The place has a special association with the life or work of a particular person, group or organisation of importance in Queensland&apos;s history.
Since 1926, the place has been associated with the work of the Catholic Church in femal

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
+ 空文字列となったトークンは削除  

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．



In [5]:
def tokenizer(text):
    del_token = '.,!?;:()[]\'"'
    return ' '.join(i.strip(del_token) for i in text.strip().split() if i.strip(del_token))

In [6]:
def tokenize_file(input_file, output_file):
    # ファイルがすでに存在するか確認
    if not os.path.isfile(output_file):
        
        with open(input_file) as fi, open(output_file, "w") as fo:
            
            # ファイル各行ごとにtokenize
            for text in fi:
                tokens = tokenizer(text)
                
                # 空文字でなければ書き込む
                if tokens:
                    print(tokens, file=fo)

In [7]:
tokenize_file('work/enwiki-20150112-400-r100-10576.txt', 'work/r100corpus.txt')

### 確認

In [8]:
!head -n 3 'work/r100corpus.txt'

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．  


インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

[使った国名リスト](http://www.jal.co.jp/5931/readme/code.html)

In [9]:
import pandas as pd

In [10]:
url = "http://www.jal.co.jp/5931/readme/code.html"
table = pd.read_html(url)[0]

### 国名リストの確認

In [11]:
table.head()

,0,1,2
0,国/地域名,英語名,国/地域名コード
1,アイスランド,Iceland,ISL
2,アイルランド,Ireland,IRL
3,アゼルバイジャン,Azerbaijan,AZE
4,アフガニスタン,Afghanistan,AFG


### 2語以上からなる国名の (old, new) リストを作成

In [12]:
def iter_replace_country(countries):
    for country in countries:
        old = tokenizer(country)
        new = old.replace(' ', '_')
        length = len(old.split())
        
        if length > 1:
            yield (length, old, new)

In [13]:
# 長さで降順にソート
replace_countries = [(old, new) for _, old, new in sorted(iter_replace_country(table[1][1:]), key=lambda x: -x[0])]

### 上のリストでreplace

In [14]:
def combined_country(text):
    for old, new in replace_countries:
        text = text.replace(old, new)
        
    return text

In [15]:
def combined_country_file(input_file, output_file):
    # ファイルがすでに存在するか確認
    if not os.path.isfile(output_file):
        
        with open(input_file) as fi, open(output_file, 'w') as fo:
            for text in fi:
                fo.write(combined_country(text))

### テスト

In [16]:
test = 'New Zealand, United States, South Africa, Saudi Arabia, United States, Papua New Guinea and Isle of Man'

In [17]:
print(test)
print(tokenizer(test))
print(combined_country(tokenizer(test)))

New Zealand, United States, South Africa, Saudi Arabia, United States, Papua New Guinea and Isle of Man
New Zealand United States South Africa Saudi Arabia United States Papua New Guinea and Isle of Man
New_Zealand United_States South_Africa Saudi_Arabia United_States Papua_New_Guinea and Isle_of_Man


### 実行

In [18]:
combined_country_file('work/r100corpus.txt', 'work/r100corpus_combined_country.txt')

In [19]:
!cat 'work/r100corpus_combined_country.txt' | grep 'United_States' | head -n 3

The term anarchism is a compound word composed from the word anarchy and the suffix -ism themselves derived respectively from the Greek i.e anarchy from anarchos meaning one without rulers from the privative prefix ἀν- an- i.e without and archos i.e leader ruler cf archon or arkhē i.e authority sovereignty realm magistracy and the suffix or -ismos -isma from the verbal infinitive suffix -ίζειν -izein The first known use of this word was in 1539."Anarchist was the term adopted by Maximilien de Robespierre to attack those on the left whom he had used for his own ends during the French Revolution but was determined to get rid of though among these anarchists there were few who exhibited the social revolt characteristics of later anarchists There would be many revolutionaries of the early nineteenth century who contributed to the anarchist doctrines of the next generation such as William Godwin and Wilhelm Weitling but they did not use the word anarchist or anarchism in describing themselv

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．  

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）  
- 単語$t$を選ぶ度に，文脈幅$d$は{${1, 2, 3, 4, 5}$}の範囲でランダムに決める．  


In [20]:
import random

In [21]:
def extract_context(text):
    terms = text.rstrip("\n").split()
                
    # termごとにdを1~5からランダムに決め、contextを取り出す
    for i, t in enumerate(terms):
        d = random.randint(1,5)
        context_terms = terms[max(0,i-d) : i] + terms[i+1 : i+1+d]
        
        for c in context_terms:
            yield t, c

In [22]:
def extract_context_file(input_file, output_file):
    # ファイルがすでに存在するか確認
    if not os.path.isfile(output_file):
        
        with open(input_file) as fi, open(output_file, 'w') as fo:
            for text in fi:
                for t, c in extract_context(text):
                    print('{}\t{}'.format(t, c), file=fo)

### テスト

In [1]:
from itertools import islice

In [24]:
test = 'Minecraft is a dangerous game that causes time sense to go wrong'

In [25]:
for t, c in islice(extract_context(test), 10):
    print(t, c)

Minecraft is
Minecraft a
Minecraft dangerous
is Minecraft
is a
is dangerous
a Minecraft
a is
a dangerous
a game


### 実行

In [26]:
extract_context_file('work/r100corpus_combined_country.txt', 'work/r100context.txt')

In [27]:
!head 'work/r100context.txt'

Anarchism	is
Anarchism	a
Anarchism	political
Anarchism	philosophy
Anarchism	that
is	Anarchism
is	a
is	political
is	philosophy
is	that


In [28]:
!wc 'work/r100context.txt' -l

68086580 work/r100context.txt


# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,*)$: 単語$t$の出現回数
- $f(*,c)$: 文脈語$c$の出現回数
- $N$: 単語と文脈語のペアの総出現回数

In [2]:
from collections import defaultdict
from tqdm import tqdm
import pickle

In [30]:
def get_co_occur_freq(input_file):
    f_tc = defaultdict(int)
    f_t = defaultdict(int)
    f_c = defaultdict(int)
    N = 0
    
    with open(input_file) as fi:
        prev_t = ''
        for line in tqdm(fi):
            t, c = line.rstrip('\n').split('\t', 1)

            # f(t, c), f(c), Nを記録
            f_tc[(t, c)] += 1
            f_t[t] += 1
            f_c[c] += 1
            N += 1
            
    return f_tc, f_t, f_c, N

### pickleで保存
- jsonはkeyをtupleで保存できない
    - {('Anarchism', 'is') : 10} は不可
- 工夫が必要
    - {'key' : ('Anarchism', 'is'), 'value' : 10}
        - keyとvalueで分ける
    - {'Anarchism is' : 10}
        - スペースまたはタブで区切る
- pickleの場合
    - keyがtupleでも可
    - defaultdictの引数にlambdaを使うとエラー

In [31]:
def write_co_occur_freq_file(input_file, output_file, overwrite=False):
    # ファイルがすでに存在するか確認
    if not os.path.isfile(output_file) or overwrite:
        
        with open(output_file, 'wb') as fo:
            # 各それぞれを計算
            f_tc, f_t, f_c, N = get_co_occur_freq(input_file)

            # ファイルに保存
            print('saving the file now...')
            pickle.dump({'f_tc':f_tc, 'f_t':f_t, 'f_c':f_c, 'N':N}, fo)
            print('complete! The file was saved to "{}"'.format(output_file))
            
    else:
        print('"{}" already exist.'.format(output_file))

In [32]:
write_co_occur_freq_file('work/r100context.txt' ,'work/r100freq.pickle')

"work/r100freq.pickle" already exist.


↑2分50秒

### memo
- メモリを消費しない為には
    - 82のファイルを単語でソート
    - ソートするときはファイルを分割してマージソート

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

- $f(t, c) \geq 10$ならば，$X_{tc} = {\rm PPMI}(t, c) = \max\{\log \frac{N \times f(t,c)}{f(t,*) \times f(*,c)}, 0\}$
- $f(t, c) < 10$ならば，$X_{tc} = 0$  

ここで，${\rm PPMI}(t, c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [3]:
import numpy as np
from scipy import sparse
import json

In [4]:
matrix_file = 'work/r100_compressed_matrix.pickle'

In [35]:
if not os.path.isfile(matrix_file):
    # 読み込み
    with open('work/r100freq.pickle', 'rb') as fi:
        freq_dict = pickle.load(fi)

    f_tc = freq_dict['f_tc']
    f_t = freq_dict['f_t']
    f_c = freq_dict['f_c']
    N = freq_dict['N']
    X = sparse.lil_matrix((len(f_t), len(f_c)))
    word2index = defaultdict(lambda:len(word2index))

    # matrix作成
    for t, c in f_tc:
        if f_tc[(t, c)] >= 10:
            ppmi = max(np.log(N * f_tc[(t, c)] / (f_t[t] * f_c[c])), 0)
            X[word2index[t], word2index[c]] = ppmi
            
    # word2indexを保存
    with open('work/word2index.json', 'w') as fo:
        json.dump(word2index, fo)

### 確認

In [36]:
# print(X.shape)
# X.todense()

(383349, 383349)


matrix([[0.        , 0.58390874, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.58207298, 0.        , 0.28614027, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.30470806, 1.38357106, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

### memo
scipy.sparse : 種類によって、structureが違う  
- coo, lil, dok : 1つづつ要素を入力するのに便利、算術演算は不可or遅い  
- csc, csr : 算術演算は早い、要素挿入などは遅い  

- [coo](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix)
- [lil](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html#scipy.sparse.lil_matrix)
- [dok](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html#scipy.sparse.dok_matrix)
- [csc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html#scipy.sparse.csc_matrix)
- [csr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [37]:
from sklearn.decomposition import TruncatedSVD

In [38]:
if not os.path.isfile(matrix_file):
    # SVDで300dimに次元圧縮
    compressed_matrix = TruncatedSVD(300).fit_transform(X)
    
    # 保存
    with open(matrix_file, 'wb') as fo:
        pickle.dump(compressed_matrix, fo)

### 確認

In [39]:
print(compressed_matrix.shape)
compressed_matrix

(383349, 300)


array([[ 10.8240754 ,  -9.74378182,  13.94872879, ...,   0.42175619,
         -0.13219345,  -0.52398254],
       [ 13.31035638, -10.96632689,  21.68599975, ...,  -0.21242377,
          0.31986328,   0.05241682],
       [  2.88213064,  -2.34753214,   0.50170894, ...,   0.41869774,
         -1.16036486,   1.22787941],
       ...,
       [  0.        ,  -0.        ,   0.        , ...,   0.        ,
          0.        ,  -0.        ],
       [  0.        ,  -0.        ,   0.        , ...,   0.        ,
          0.        ,  -0.        ],
       [  0.        ,  -0.        ,   0.        , ...,   0.        ,
          0.        ,  -0.        ]])

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [5]:
with open(matrix_file, 'rb') as fi:
    matrix = pickle.load(fi)
    
with open('work/word2index.json') as fi:
    word2index = json.load(fi)

↓ word2indexは f_tc >= 10 の 値のみを保持しているため、word数が少ない

In [6]:
print('matrix.shape = {}'.format(matrix.shape))
print('len(word2index) = {}'.format(len(word2index)))

matrix.shape = (383349, 300)
len(word2index) = 34073


In [7]:
matrix[word2index['United_States']]

array([ 3.67042754e+00, -7.97679639e-01,  4.83658640e-01,  6.46441761e+00,
        2.33432846e+00, -1.20769317e+00, -2.31089079e+00,  3.48054571e-01,
        2.97345403e-02,  5.00593945e-01, -2.47616574e-01,  1.26203295e+00,
       -2.59763758e+00, -2.64483439e+00,  2.46400364e+00,  2.34843206e+00,
       -3.29193245e+00,  6.85065832e-01,  2.67604669e+00,  4.55965011e-01,
       -1.23002647e-01,  7.40697015e-01,  1.07810134e-01, -1.06869851e+00,
        2.90179984e-01, -1.61695332e+00,  9.59575152e-01,  3.65895272e+00,
        6.08389216e-01, -4.31464005e+00, -8.07831242e-01,  1.77890964e-01,
       -8.89742180e-02,  1.15314676e+00,  9.68645229e-01, -4.88669318e-01,
        3.02646083e+00,  6.52261726e-01,  1.76830870e+00,  1.30764205e+00,
        1.24047947e-03, -8.84145208e-01, -2.55120934e+00, -1.43524336e+00,
        4.14247851e-01, -3.80612089e-01, -4.16792161e-02,  9.13717867e-01,
        2.30521373e-01,  9.88985700e-01,  1.91767517e+00, -2.06546754e-01,
        1.15842955e+00, -

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [8]:
def cos_sim(v1, v2): 
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-12)

In [9]:
v1 = matrix[word2index['United_States']]
v2 = matrix[word2index['U.S']]
cos_sim(v1, v2)  # vec同士の演算

0.8068498000328898

### または

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
v1 = v1.reshape(1, -1)
v2 = v2.reshape(1, -1)
cosine_similarity(v1, v2)  # matrix同士の演算

array([[0.8068498]])

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

### 1. 自作cosで

In [12]:
def similar1_topn(vec, matrix, word2index, start=1, end=11):
    word_sim = dict()
    for w, idx in word2index.items():
        word_sim[w] = cos_sim(matrix[idx], vec)

    return {w : s for w, s in islice(sorted(word_sim.items(), key=lambda x: -x[1]), start, end)}

In [13]:
similar1_topn(matrix[word2index['England']], matrix, word2index)

{'Scotland': 0.7035333633555668,
 'Spain': 0.5949160803484714,
 'Australia': 0.5813180337813137,
 'Wales': 0.5561339431439319,
 'France': 0.5553863308259942,
 'Italy': 0.5489628858410673,
 'Germany': 0.5370692279123737,
 'Ireland': 0.5289999988115132,
 'United_Kingdom': 0.5176723042774007,
 'Britain': 0.5074796089614494}

### 2. sklearnのcosで

In [14]:
def similar2_topn(vec, matrix, word2index, start=1, end=11):
    sim_vec = cosine_similarity(vec.reshape(1, -1), matrix)[0]
    sim_indices = np.argsort(sim_vec)[::-1][start:end]
    index2word = {v : k for k, v in word2index.items()}
    
    return {index2word[i] : sim_vec[i] for i in sim_indices}

In [15]:
similar2_topn(matrix[word2index['England']], matrix, word2index)

{'Scotland': 0.7035333633555734,
 'Spain': 0.5949160803484759,
 'Australia': 0.5813180337813166,
 'Wales': 0.5561339431439363,
 'France': 0.5553863308259964,
 'Italy': 0.548962885841071,
 'Germany': 0.5370692279123762,
 'Ireland': 0.5289999988115169,
 'United_Kingdom': 0.5176723042774056,
 'Britain': 0.5074796089614539}

### 3. 2の改良ver (演算する行列を絞る)

In [16]:
def similar3_topn(vec, matrix, word2index, start=1, end=11):
    # word2indexに登場するものにmatrixを絞る (次元数 : 38万 -> 3万)
    indices = np.array([i for i in word2index.values()])
    matrix = matrix[indices]
    
    # 演算
    sim_vec = cosine_similarity(vec.reshape(1, -1), matrix)[0]
    sim_indices = np.argsort(sim_vec)[::-1][start:end]
    index2word = {v : k for k, v in word2index.items()}
    
    return {index2word[i] : sim_vec[i] for i in sim_indices}

In [17]:
similar3_topn(matrix[word2index['England']], matrix, word2index)

{'Scotland': 0.7035333633555734,
 'Spain': 0.5949160803484759,
 'Australia': 0.5813180337813166,
 'Wales': 0.5561339431439363,
 'France': 0.5553863308259964,
 'Italy': 0.548962885841071,
 'Germany': 0.5370692279123762,
 'Ireland': 0.5289999988115169,
 'United_Kingdom': 0.5176723042774056,
 'Britain': 0.5074796089614539}

↑ くそ早い

# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [18]:
vec = matrix[word2index['Spain']] - matrix[word2index['Madrid']] + matrix[word2index['Athens']]

In [19]:
similar3_topn(vec, matrix, word2index, start=0, end=10)

{'Spain': 0.9385985291752981,
 'France': 0.8538903741649336,
 'Sweden': 0.836090676998262,
 'Italy': 0.8288329327777637,
 'Germany': 0.8027078157007357,
 'Netherlands': 0.8025955806250862,
 'Austria': 0.8014099452237468,
 'Portugal': 0.7770318968921359,
 'Télévisions': 0.7711391042620539,
 'Denmark': 0.7696654238787894}